In [2]:
import os
import re
import csv
import glob
import time
from collections import OrderedDict
import pandas as pd

## read Category:Japanese Female Idol (日本の女性アイドル）articles data downloaded from PetScan (https://petscan.wmflabs.org/?psid=22392796)
### User manual (https://meta.wikimedia.org/wiki/PetScan/)
-  read download.csv (日本の女性アイドル(Japanese male idol|1　birth year|3　people|10)

In [3]:
df = pd.read_csv("japaneseFemaleIdol.csv")
df.head(20)

,number,title,pageid,namespace,length,touched
0,1,松田聖子,642,NaN,192226,20220702101440
1,2,内田有紀,830,NaN,37777,20220630231616
2,3,後藤久美子,840,NaN,26714,20220523150020
3,4,吉田里深,855,NaN,3862,20210309112016
4,5,吉岡美穂,4967,NaN,15463,20220630080705
5,6,市井紗耶香,15889,NaN,23378,20220702033344
6,7,滝沢乃南,16565,NaN,13714,20220417110345
7,8,優香,16620,NaN,46152,20220505150604
8,9,宍戸留美,17848,NaN,51712,20220701162618
9,10,松浦亜弥,18735,NaN,64010,20220615040200


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5108 entries, 0 to 5107
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   number     5108 non-null   int64  
 1   title      5108 non-null   object 
 2   pageid     5108 non-null   int64  
 3   namespace  0 non-null      float64
 4   length     5108 non-null   int64  
 5   touched    5108 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 239.6+ KB


In [5]:
# key: folder's name, value: create dictionartris of max IDs
# exrtract max ID values from the last article in the last file in each directory
def mapping_dir(dir_path):
    dir_list = os.listdir(dir_path)
    for _dir in dir_list:
        last_f_path = sorted(glob.glob(os.path.join(dir_path, _dir) + '\\*'))[-1]
        max_id, _, _ = format_text(sorted(get_articlelist(last_f_path))[-1])
        dir_info[_dir] = max_id
    return dir_info


In [6]:
# pade_id from csv file and dir_info created by mapping_dir
# return folder name which include page_id I'm looking for
def get_search_dir(page_id, dir_info):
    min_id = 0
    for dir_name, max_id in dir_info.items():
        search_dir = dir_name
        if min_id <= page_id <= max_id:
            return search_dir
        else:
            min_id = max_id
    return next(iter(dir_info))     # if not found, retuen the first value(AA)
 

In [7]:
# If nothing is found with search_dir, search again from the beginning
# this can be replaced by "get_files"
def get_all_files(base_path, search_dir):
    dir_iter = iter(dir_info.keys())   # in case search_dir fails
    dir_list = list(dir_info.keys())   # list of key
    cnt = 1
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        search_dir = next(dir_iter)
        cnt += 1
        if search_dir == dir_list[-1] and cnt == len(dir_list):
            # when nothing found
            return

In [8]:
# 
def get_files(base_path, search_dir):
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        else:
            return

In [9]:
# return each article text
def format_text(data):
    head = head_reg.search(data)  # (<doc id=...>)
    id_tag = id_reg.search(head.group())
    doc_id = re.search(r'\d+', id_tag.group())  # extract id from header
    title_tag = title_reg.search(head.group())
    doc_title = re.search(r'".+"', title_tag.group())  
    doc_id = int(doc_id.group())
    title = doc_title.group().replace('"', '')      # delete ""
    # deete each header(<doc id=...>)and tag
    # extract only texts (delete title as some articles have duplicated tittles)
    text = data.replace(head.group(), '').rstrip('</doc>')[len(title):]
    return doc_id, title, text

In [10]:
# return lists of articles
def get_articlelist(f_path):
    f = open(f_path, 'r', encoding='utf-8')
    file_data = f.read()
    file_data = file_data.replace('\n', '')
    return doc_reg.findall(file_data)

In [11]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [12]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [13]:
if __name__ == '__main__':
    # 
    id_reg = re.compile(r'id="\d+"')
    title_reg = re.compile(r'title=".+"')
    doc_reg = re.compile(r'<doc id=.+?</doc>')
    head_reg = re.compile(r'<doc id.+?">')

In [14]:
# NEED TO CHANGE for differendt data
BASE_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor'
CSV_FILE = BASE_PATH + '/japaneseFemaleIdol.csv' # downloaded data from PetScan
WIKI_DIR = BASE_PATH + '/text'
OUTPUT_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor/japaneseFemaleIdol/'

In [15]:
# make dictionary of the max ID value in each folder
dir_info = OrderedDict()
dir_info = mapping_dir(WIKI_DIR)
counter = 1     # loop counter

In [16]:
# read csv
f = open(CSV_FILE, 'r', encoding='utf-8')
reader = csv.reader(f)
next(csv.reader(f))

['number', 'title', 'pageid', 'namespace', 'length', 'touched']

In [17]:
not_found = {}      # dict for not-found data

In [18]:
# measure time
start_time = time.time()

In [19]:
for line in reader:
        flag = False
        page_name = line[1]
        page_id = int(line[2])
        namespace = line[3]
        if namespace == "Category" or namespace == "Template":
            continue
        search_dir = get_search_dir(page_id, dir_info)
        for f_path in get_files(WIKI_DIR, search_dir):
            for d in get_articlelist(f_path):
                doc_id, title, text = format_text(d)
                if page_id == doc_id:
                    print(counter, title)
                    output_file(title, text)
                    counter += 1
                    flag = True
                    break
            if flag:
                break
        else:
            not_found[page_id] = page_name
            print("ページID: {}は見つかりませんでした.".format(page_id))
 
        end_time = time.time()
        processing_time = end_time - start_time
        print('processing_time(sec): ', processing_time)
 
        # print "not found" page and its ID
        for i, (_id, _title) in enumerate(not_found.items()):
            print(i, _id, _title)

F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
1 松田聖子
processing_time(sec):  1.2892606258392334
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
2 内田有紀
processing_time(sec):  1.3438057899475098
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
3 後藤久美子
processing_time(sec):  1.3860411643981934
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
4 吉田里深
processing_time(sec):  1.4330787658691406
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
5 吉岡美穂
processing_time(sec):  1.7338085174560547
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
6 市井紗耶香
processing_time(sec):  2.558527946472168
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
7 滝沢乃南
processing_time(sec):  3.4205243587493896
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
8 優香
processing_time(sec):  4.248795509338379
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
9 宍戸留美
processing_time(sec):  4.342549562454224
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
10

78 相本久美子
processing_time(sec):  30.062570333480835
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
79 大場久美子
processing_time(sec):  30.983157634735107
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
80 蓮舫
processing_time(sec):  31.977606058120728
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
81 松本明子
processing_time(sec):  33.75898885726929
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
82 あべ静江
processing_time(sec):  35.74173545837402
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
83 青山みなみ
processing_time(sec):  37.52568078041077
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
84 亜希
processing_time(sec):  39.90817356109619
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
85 石村舞波
processing_time(sec):  41.359580755233765
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
86 熊井友理奈
processing_time(sec):  42.484084367752075
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
87 須藤茉麻
processing_time(sec):  43.62289476394653
F

156 井上望
processing_time(sec):  88.30103349685669
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
157 中嶋ミチヨ
processing_time(sec):  89.10303735733032
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
158 田村英里子
processing_time(sec):  89.84570550918579
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
159 田中陽子 (タレント)
processing_time(sec):  90.55143356323242
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
160 橋本マナミ
processing_time(sec):  91.30406045913696
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
161 江川有未
processing_time(sec):  92.10207104682922
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
162 河合その子
processing_time(sec):  92.88057851791382
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
163 下川みくに
processing_time(sec):  93.6870710849762
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
164 山本ゆか里
processing_time(sec):  94.49791359901428
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
165 斉藤慶子
processing_time(sec):  95.

233 青沼ちあさ
processing_time(sec):  140.99546480178833
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
234 甲斐智枝美
processing_time(sec):  141.7795352935791
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
235 虎南有香
processing_time(sec):  142.6495442390442
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
236 我妻三輪子
processing_time(sec):  143.60332942008972
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
237 森脇ゆか
processing_time(sec):  144.4855020046234
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
238 池永亜美
processing_time(sec):  145.34780073165894
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
239 谷口紗耶香
processing_time(sec):  146.18578457832336
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
240 鈴田林沙
processing_time(sec):  147.00628447532654
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
241 佐野光来
processing_time(sec):  147.9479820728302
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
242 大森玲子
processing_time(sec):  148.8

310 岡あゆみ
processing_time(sec):  193.99444437026978
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
311 久志麻理奈
processing_time(sec):  194.7558560371399
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
312 城生綾菜
processing_time(sec):  195.52798438072205
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
313 可愛かずみ
processing_time(sec):  196.35660576820374
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
314 佐々木絵美子
processing_time(sec):  197.2733278274536
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
315 藤本綾
processing_time(sec):  198.0034143924713
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
316 大瀧彩乃
processing_time(sec):  198.7908742427826
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
317 森ひろこ
processing_time(sec):  199.6492462158203
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
318 小林裕美
processing_time(sec):  200.39979434013367
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
319 町田恵 (タレント)
processing_time(sec):  2

...

4826 神田ジュナ
processing_time(sec):  4896.81679558754
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4827 河路由希子
processing_time(sec):  4898.068453073502
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4828 葉月にこ
processing_time(sec):  4899.4080131053925
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4829 斎藤恭代
processing_time(sec):  4901.249501943588
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4830 月村麗華
processing_time(sec):  4902.713449716568
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4831 新海まき
processing_time(sec):  4904.179023504257
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4832 前田美咲
processing_time(sec):  4905.621249437332
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4833 田村菜月
processing_time(sec):  4907.433842897415
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4834 近藤ありす
processing_time(sec):  4909.639967441559
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
4835 溝口亜以子
processing_time(sec):  

4906 岡本杷奈
processing_time(sec):  5036.7551445961
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4907 泉舞子
processing_time(sec):  5036.797916173935
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4908 太陽 (アイドル)
processing_time(sec):  5036.837905406952
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4909 萌木七海
processing_time(sec):  5036.913032770157
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4910 結城りな
processing_time(sec):  5037.01721072197
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4911 葵るり
processing_time(sec):  5037.113476991653
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4912 合田柚奈
processing_time(sec):  5037.209523439407
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4913 河内麻沙美
processing_time(sec):  5037.326785087585
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4914 吉国あゆみ
processing_time(sec):  5037.4476583004
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4915 伊藤綾香 (アイドル)
processing_time(sec)

4983 小川彩
processing_time(sec):  5067.148230075836
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4984 冨里奈央
processing_time(sec):  5067.891717433929
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4985 金子みゆ
processing_time(sec):  5068.617963075638
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4986 中西アルノ
processing_time(sec):  5069.506727933884
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4987 平崎里奈
processing_time(sec):  5070.174823999405
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4988 小鳥遊くれあ
processing_time(sec):  5070.848465919495
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4989 戸塚こはる
processing_time(sec):  5071.509947776794
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4990 鈴木Mob.
processing_time(sec):  5072.119056463242
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4991 五百城茉央
processing_time(sec):  5072.731178760529
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
4992 志田こはく
processing_time(sec)

5060 高坂琴水
processing_time(sec):  5136.435048818588
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5061 深見望乃
processing_time(sec):  5137.607213258743
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5062 音海まい
processing_time(sec):  5138.741771697998
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5063 Nonamera
processing_time(sec):  5139.790456771851
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5064 前園あやの
processing_time(sec):  5140.879261732101
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5065 ごいちー
processing_time(sec):  5141.961457967758
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5066 桜田あゆみ
processing_time(sec):  5143.090683698654
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5067 拍羽想
processing_time(sec):  5144.2659821510315
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5068 辻村ゆりな
processing_time(sec):  5146.427748680115
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
5069 Tnaka
processing_time(sec

ページID: 4617642は見つかりませんでした.
processing_time(sec):  5218.945622682571
0 4603797 広山楓
1 4603810 上村梨々香
2 4603812 森朱里
3 4603814 佐藤妃希
4 4605694 南みゆか
5 4605696 高橋ピーチももか
6 4605699 KAHO
7 4605700 YURINA
8 4605702 原田咲
9 4605888 笹木里緒菜
10 4607302 市原薫
11 4607732 城田理加
12 4607803 RЯ
13 4608797 乙陽葵
14 4608828 青木胡杜音
15 4611394 日比野芽奈
16 4613862 大葉みらい
17 4615947 大塚杏奈
18 4616631 茅野りお
19 4616992 矢野美優
20 4617205 桃瀬エル
21 4617642 大曲李佳
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
ページID: 4618156は見つかりませんでした.
processing_time(sec):  5224.479849100113
0 4603797 広山楓
1 4603810 上村梨々香
2 4603812 森朱里
3 4603814 佐藤妃希
4 4605694 南みゆか
5 4605696 高橋ピーチももか
6 4605699 KAHO
7 4605700 YURINA
8 4605702 原田咲
9 4605888 笹木里緒菜
10 4607302 市原薫
11 4607732 城田理加
12 4607803 RЯ
13 4608797 乙陽葵
14 4608828 青木胡杜音
15 4611394 日比野芽奈
16 4613862 大葉みらい
17 4615947 大塚杏奈
18 4616631 茅野りお
19 4616992 矢野美優
20 4617205 桃瀬エル
21 4617642 大曲李佳
22 4618156 櫻井梨央
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
ページID: 4618158は見つかりませんでした.
processing_time(s